<center> <h1> Diagnose Latent Cytomegalovirus Using CyTOF Data and Deep Learning <h1> </center>

<center>Zicheng Hu, Ph.D.</center>
<center>Research Scientist</center>
<center>ImmPort Team</center>
<center>The Unversity of California, San Francisco</center>

![alt text](Data/header.png)

### Introduction
A deep neural network (a.k.a. deep learning) is an artificial neural network with multiple layers between the input and output layers. It was proven to be highly effective for a variety of predictive tasks. In health care, deep learning is quickly gaining popularity and has been implemented for applications such as image-based diagnosis and personalized drug recommendations. In this tutorial, we will build a tailored deep-learning model for CyTOF data to diagnosis latent Cytomegalovirus infection using Keras and TensorFlow. To run this tutorial, download the [github repository](https://github.com/hzc363/DeepLearningCyTOF) and run the [jupyter notebook](https://github.com/hzc363/DeepLearningCyTOF/blob/master/DeepLearning_CyTOF.ipynb). 

### Step 1: Import Functions
Before we start, we first import functions that we will use in this tutorial from different libraries. 

In [1]:
##### Step 1: import functions #####
import tensorflow as tf
from tensorflow.keras.layers import Dense, Flatten, BatchNormalization, Activation, Conv2D, AveragePooling2D, Input, Softmax
from tensorflow.keras.models import load_model, Model, Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras import backend as K
from tensorflow.keras.utils import plot_model
import pickle
import pandas as pd
import numpy as np
from numpy.random import seed; seed(111)
import random
import matplotlib.pyplot as plt
import seaborn as sns
from tensorflow.random import set_seed; set_seed(111)
from sklearn.metrics import roc_curve, auc
from six import StringIO  
from sklearn.tree import export_graphviz, DecisionTreeRegressor
from sklearn.model_selection import train_test_split
from scipy.stats import ttest_ind
from IPython.display import Image  
import pydotplus
import time

In [2]:
gpus = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(gpus[0], True)

IndexError: list index out of range

In [ ]:
phenotype = 'Expsample Treatement'

### Step 2: Load data

We load the data, which are stored in the "allData.obj" file. The data includes three parts, meta-data, CyTOF data, and marker names. 

* The **CyTOF data** contains the single-cell profile of 27 markers. For the convenience of this tutorial, we already downloaded the fcs files from ImmPort and preprocessed the data into Numpy arrays. See [an example](https://github.com/hzc363/DeepLearningCyTOF/tree/master/FCS_to_Array) for the preprocessing of the FCS files. The dimension of the Numpy array is 472 samples x 10000 cells x 27 markers.
* The **metadata** contains the sample level information, including the study accession number for each sample and the ground truth of CMV infection. It is stored as a pandas data frame.
* The **marker names** contain the name of the 27 markers.

In [4]:
##### Step 2: load data #####

tutorial_files = ! ls Data
if "allData.obj" not in tutorial_files:
    print("Downloading Data:")
    ! wget https://ucsf.box.com/shared/static/tn8m9hco3f3gugn7z394yzimvtfdg129.obj -O ./Data/allData.obj
        
#load data
with open("FCS_to_Array/agingData.obj", "rb") as f:
    allData = pickle.load(f)
metadata = allData["samples"].reset_index().drop(columns=['Unnamed: 0','index'])
cytoData = allData['expr_list']
markerNames = allData["marker_names"]

print("Dimentions of metadata: ", metadata.shape)
print("Dimensions of cytoData: ",cytoData.shape,"\n")
print("Names of the makers: \n",markerNames.values)

Dimentions of metadata:  (753, 46)
Dimensions of cytoData:  (753, 20000, 13, 1) 

Names of the makers: 
 ['FSC-A' 'FSC-H' 'FSC-W' 'SSC-A' 'SSC-H' 'SSC-W' 'PSTAT1' 'PSTAT5' 'CD66B'
 'CD33' 'CD3/CD14' 'PSTAT3' 'CD4/CD19']


In [5]:
metadata[phenotype].value_counts()

IL-10               154
IL-6                151
interferon alpha    150
IL-21               149
Unstim              149
Name: Expsample Treatement, dtype: int64

In [6]:
all_metadata = metadata
all_cytodata = cytoData
metadata = metadata[metadata[phenotype].isin(('Unstim','IL-10'))]

In [7]:
cytoData = []
for i, row in metadata.iterrows():
    if row[phenotype] in {'Unstim','IL-10'}:
        cytoData.append(all_cytodata[i])
cytoData = np.asarray(cytoData)

### Step 3: Split data into training, validation and testing sets
Now, lets split the data into training, validation, and testing sets. The training data is used to train the deep learning model. The validation dataset is used to select the best parameters for the model and to avoid overfitting. The test dataset is used to evaluate the performance of the final model.

The CyTOF dataset contains samples from 9 studies available on ImmPort. We will use samples from the study SDY515 as a validation set, samples from the study SDY519 as a testing set, and the rest of the samples as a training set. 

In [8]:
##### Step 3: split train, validation and test######
phenos = sorted(set(metadata[phenotype].values))
y = np.array([phenos.index(_) for _ in metadata[phenotype].values])
y = y.reshape(list(y.shape)+[1])
x = cytoData
pheno_map = dict(enumerate(phenos))

x_train, x_valid, y_train, y_valid = train_test_split(x, y)
# train_id = (metadata.study_accession.isin(["SDY515","SDY519"])==False)
# valid_id = metadata.study_accession=="SDY515"
# test_id = metadata.study_accession =="SDY519"

# x_train = x[train_id]; y_train = y[train_id]
# x_valid = x[valid_id]; y_valid = y[valid_id]
# x_test = x[test_id]; y_test = y[test_id]

In [9]:
y.shape

(303, 1)

In [10]:
list(metadata[phenotype].values) == [pheno_map[_[0]] for _ in y]

True

### Step 4: Define the deep learning model
We will use a customized convolution neural network (CNN) to analyze the CyTOF data. For each sample, the CyTOF data is a matrix with rows as cells and columns as markers. It is crucial to notice that the CyTOF data is an unordered collection of cells (rows). For example, both matrix 1 and matrix 2 profiles the same sample in Figure 1A, even though they have different orders of rows. 

Based on the characteristics of the CyTOF data, we design a CNN model that is invariant to the permutation of rows. The model contains six layers: input layer, first and second convolution layer, pooling layer, dense layer, and output layer. 

* The **input layer** receives the CyTOF data matrix. 

* The **first convolution layer** uses three filters to scan each row of the CyTOF data. This layer extracts relevant information from the cell marker profile of each cell. 

* The **second convolution layer** uses three filters to scan each row of the first layer's output. Each filter combines information from the first layer for each cell. 

* The **pooling layers** averages the outputs of the second convolution layer. The purpose is to aggregate the cell level information into sample-level information. 

* The **dense layer** further extracts information from the pooling layer. 

* The **output layer** uses logistic regression to report the probability of CMV infection for each sample. 


In [11]:
##### Step 4: define model #####

# input
model_input = Input(shape=x_train[0].shape)

# first convolution layer
model_output = Conv2D(3, kernel_size=(1, x_train.shape[2]),
                 activation=None)(model_input)
model_output = BatchNormalization()(model_output)
model_output = Activation("relu")(model_output)

# sceond convolution layer
model_output = Conv2D(3, (1, 1), activation=None)(model_output)
model_output = BatchNormalization()(model_output)
model_output = Activation("relu")(model_output)

# pooling layer
model_output = AveragePooling2D(pool_size=(x_train.shape[1], 1))(model_output)
model_output = Flatten()(model_output)

# Dense layer
model_output = Dense(3, activation=None)(model_output)
model_output = BatchNormalization()(model_output)
model_output = Activation("relu")(model_output)

# output layer
model_output = Dense(1, activation=None)(model_output)
model_output = BatchNormalization()(model_output)
model_output = Activation("sigmoid")(model_output)

##### Step 5: Fit model #####

# specify input and output
example_model = Model(inputs=[model_input],
              outputs=model_output)

# define loss function and optimizer
example_model.compile(loss='binary_crossentropy',
              optimizer=Adam(lr=0.0001),
              metrics=['accuracy'])

example_model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 20000, 13, 1)]    0         
_________________________________________________________________
conv2d (Conv2D)              (None, 20000, 1, 3)       42        
_________________________________________________________________
batch_normalization (BatchNo (None, 20000, 1, 3)       12        
_________________________________________________________________
activation (Activation)      (None, 20000, 1, 3)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 20000, 1, 3)       12        
_________________________________________________________________
batch_normalization_1 (Batch (None, 20000, 1, 3)       12        
_________________________________________________________________
activation_1 (Activation)    (None, 20000, 1, 3)       0     

In [12]:
x.shape

(303, 20000, 13, 1)

In [13]:
aging_model = Sequential([
    Input(shape=x[0].shape),
    Conv2D(12, kernel_size = (1, x.shape[2]), activation=None),
#     BatchNormalization(),
    Activation('relu'),
    Conv2D(3, kernel_size = (1,1), activation=None),
#     BatchNormalization(),
    Activation('relu'),
    AveragePooling2D(pool_size = (x.shape[1], 1)),
    Flatten(),
    Dense(12, activation=None),
#     BatchNormalization(),
    Activation('relu'),
    Dense(len(pheno_map), activation=None),
#     BatchNormalization(),
#     Activation('sigmoid'),
    Softmax(),
])
aging_model.compile(loss=SparseCategoricalCrossentropy(from_logits=True),
              optimizer='adam',
              metrics=['accuracy'])

In [14]:
aging_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 20000, 1, 12)      168       
_________________________________________________________________
activation_4 (Activation)    (None, 20000, 1, 12)      0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 20000, 1, 3)       39        
_________________________________________________________________
activation_5 (Activation)    (None, 20000, 1, 3)       0         
_________________________________________________________________
average_pooling2d_1 (Average (None, 1, 1, 3)           0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 3)                 0         
_________________________________________________________________
dense_2 (Dense)              (None, 12)                4

In [15]:
model = aging_model

# save the best performing model
checkpointer = ModelCheckpoint(filepath='Result/saved_weights.hdf5', 
                               monitor='val_loss', verbose=0, 
                               save_best_only=True)

st = time.time()
# model training
model.fit([x_train], y_train,
          batch_size=60,
          epochs=100, 
          verbose=1,
          callbacks=[checkpointer],
          validation_data=([x_valid], y_valid))
rt = time.time()-st

Epoch 1/100
4/4 [==============================] - 4s 369ms/step - loss: 0.6925 - accuracy: 0.5136 - val_loss: 0.6940 - val_accuracy: 0.5132
Epoch 2/100


ResourceExhaustedError:  OOM when allocating tensor with shape[60,12,20000,1] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[node sequential/conv2d_2/Conv2D (defined at <ipython-input-15-fb344f3146ef>:10) ]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.
 [Op:__inference_train_function_1012]

Function call stack:
train_function


In [ ]:
rt

In [ ]:
##### Step 6: plot train and validation loss #####
plt.plot(model.history.history['loss'])
plt.plot(model.history.history['val_loss'])
plt.title('model train vs validation loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper right')
plt.show()

In [ ]:
##### Step 6: plot train and validation loss #####
plt.plot(model.history.history['accuracy'])
plt.plot(model.history.history['val_accuracy'])
plt.title('model train vs validation accuracy')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper right')
plt.show()

### Step 7: Evaluate the performance using test data
We load the final model from a save file (Final_weights.hdf5) for the following analysis steps. We will use the test data, which has not been touched so far, to evaluate the performance of the final model. We will draw a Receiver Operator Characteristic(ROC) Curve and use Area Under the Curve (AUC) to measure performance. 

In [ ]:
##### Step 7: test the final model #####

# load final model
# final_model = load_model('Data/Final_weights.hdf5')
final_model = model
x_test = x_valid
y_test = y_valid

# generate ROC and AUC
y_scores = final_model.predict([x_test])

In [ ]:
fpr, tpr, _ = roc_curve(y_test, y_scores)
roc_auc = auc(fpr, tpr)

# plot ROC curve
plt.plot(fpr, tpr)
plt.plot([0, 1], [0, 1], 'k--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('AUC = {0:.2f}'.format(roc_auc))
plt.show()